In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
TRAIN_PATH="/content/drive/MyDrive/Kaggle/DATA/drive/dataset_leaf_detection/train_set"
VAL_PATH="/content/drive/MyDrive/Kaggle/DATA/drive/dataset_leaf_detection/test_data"

In [17]:
for types in os.listdir(TRAIN_PATH):
    print(str(len(os.listdir(TRAIN_PATH+"/"+ types)))+" "+ types+' images')

917 Pepper__bell___Bacterial_spot images
1402 Pepper__bell___healthy images
927 Potato___Early_blight images
919 Potato___Late_blight images
127 Potato___healthy images
2017 Tomato_Bacterial_spot images
919 Tomato_Early_blight images
1816 Tomato_Late_blight images
879 Tomato_Leaf_Mold images
1682 Tomato_Septoria_leaf_spot images
1603 Tomato_Spider_mites_Two_spotted_spider_mite images
1307 Tomato__Target_Spot images
3068 Tomato__Tomato_YellowLeaf__Curl_Virus images
308 Tomato__Tomato_mosaic_virus images
1494 Tomato_healthy images


In [ ]:
###############################################################################

In [11]:
# Complete Dataset images can be loaded using ImageDataGenerator function
img_size=128
batch_size=64 #no of images in a batch
datagen_train=ImageDataGenerator(horizontal_flip=True) #generation of multiple images of same -data augmentation -tool- image data generator
train_generator=datagen_train.flow_from_directory(TRAIN_PATH, #flowing image data from correspnding dir to train generator
                            target_size=(img_size,img_size),
                            batch_size=batch_size,
                            class_mode='categorical',# bcoz we uses 15 classes 
                            shuffle=True)#to read the image random

datagen_test=ImageDataGenerator(horizontal_flip=True)
validation_generator=datagen_test.flow_from_directory(VAL_PATH,
target_size=(img_size,img_size),
batch_size=batch_size,
class_mode='categorical',
shuffle=True)

Found 19384 images belonging to 15 classes.
Found 1254 images belonging to 15 classes.


In [12]:
baseModel = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False)
inputConfig = Input(shape=(128, 128, 3))

model = baseModel(inputConfig) # customizing model size
model = GlobalAveragePooling2D()(model)
model = BatchNormalization()(model)#to reduce the overfitting
model = Dropout(0.5)(model)#to diasable random neurons
model = Dense(256, activation='relu')(model)

model = Dropout(0.5)(model)
output = Dense(15,activation = 'softmax', name='root')(model)
finalModel = Model(inputConfig,output)   
optimizer = Adam(learning_rate=0.002, epsilon=0.1, decay=0.0)
finalModel.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


29097984/29084464 [==============================] - 0s 0us/step


In [13]:
finalModel.summary()
    

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, None, None, 1024)  7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 256)               262400

In [14]:
finalModel.fit(train_generator,
                    epochs=40,
                    validation_data=validation_generator,
                )

KeyboardInterrupt: ignored

In [ ]:
finalModel.save('modeldetection.h5')


In [ ]:
##################################################################################################

In [ ]:
baseModel.summary()

In [ ]:
from keras.models import load_model

model = load_model('modeldetection.h5')